In [4]:
#Import all the libraries needed
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM,Bidirectional,Dense,Dropout # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re

In [ ]:
#Preview dataset

In [ ]:
#Declaring the english stop words

In [20]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
english_stops = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#Preprocessing and Encoding labels

In [8]:
import pathlib
from google.colab import drive
drive.mount('/content/drive')
data_train = pathlib.Path('/content/drive/My Drive/Assignment_02/phm_train.csv')
data_test = pathlib.Path('/content/drive/My Drive/Assignment_02/phm_test.csv')
data_train = pd.read_csv(data_train)
print(data_train)


Mounted at /content/drive
          tweet_id  label                                              tweet
0     6.430000e+17      0  user_mention all i can tell you is i have had ...
1     6.440000e+17      0  my doctor told me stop he gave me sum pop i mi...
2     8.150000e+17      1  i take tylenol and i wake up in the middle of ...
3     6.820000e+17      0  i got xans in an advil bottle i dont take them...
4     6.440000e+17      1  mom says i need to stop eating so much bc ive ...
...            ...    ...                                                ...
9986  6.480000e+17      1                          that vicodin messed me up
9987  5.710000e+17      0                  user_mention get some tylenol lol
9988  6.470000e+17      0                          like a walking tamiflu ad
9989  6.990000e+17      0                         klay and steph on steroids
9990  8.230000e+17      0                    horrible pops another xanax url

[9991 rows x 3 columns]


In [9]:
data_test = pd.read_csv(data_test)
print(data_train)

          tweet_id  label                                              tweet
0     6.430000e+17      0  user_mention all i can tell you is i have had ...
1     6.440000e+17      0  my doctor told me stop he gave me sum pop i mi...
2     8.150000e+17      1  i take tylenol and i wake up in the middle of ...
3     6.820000e+17      0  i got xans in an advil bottle i dont take them...
4     6.440000e+17      1  mom says i need to stop eating so much bc ive ...
...            ...    ...                                                ...
9986  6.480000e+17      1                          that vicodin messed me up
9987  5.710000e+17      0                  user_mention get some tylenol lol
9988  6.470000e+17      0                          like a walking tamiflu ad
9989  6.990000e+17      0                         klay and steph on steroids
9990  8.230000e+17      0                    horrible pops another xanax url

[9991 rows x 3 columns]


In [29]:

def preprocess(tweets):
    tweets = tweets.replace({'<.*?>': ''}, regex = True)          # remove html tag
    tweets = tweets.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    tweets = tweets.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    tweets = tweets.apply(lambda review: [w.lower() for w in review])   # lower case

    return tweets

def load_dataset():
  # tarin data
    x_train = data_train['tweet']       # Tweets/Input
    y_train = data_train['label']    # labels/Output

  # test data
    x_test = data_test['tweet']       # Tweets/Input
    y_test = data_test['label']

    x_train = preprocess(x_train)
    x_test = preprocess(x_test)


    return x_train, y_train, x_test, y_test


x_train, y_train, x_test, y_test = load_dataset()

print('Tweets:')
print(x_train, '\n')
print(x_test.head(), '\n')
print('Labels:')
print(y_train)
print(y_test.head())

Tweets:
0       [user, mention, tell, relapses, cure, hear, do...
1       [doctor, told, stop, gave, sum, pop, mix, w, a...
2       [take, tylenol, wake, middle, night, put, ice,...
3       [got, xans, advil, bottle, dont, take, shits, ...
4       [mom, says, need, stop, eating, much, bc, ive,...
                              ...                        
9986                                    [vicodin, messed]
9987                   [user, mention, get, tylenol, lol]
9988                         [like, walking, tamiflu, ad]
9989                              [klay, steph, steroids]
9990                [horrible, pops, another, xanax, url]
Name: tweet, Length: 9991, dtype: object 

0    [try, run, away, iv, needle, doctor, drug, w, ...
1    [knew, took, ambien, sleep, early, im, ready, ...
2    [mean, get, celexa, reason, behind, lot, weigh...
3    [call, dumb, dumb, one, time, dont, care, many...
4    [want, go, grocery, store, cant, pay, anyone, ...
Name: tweet, dtype: object 

Labels:

In [ ]:
#train to test split
#x_train, x_test, y_train, y_test = train_test_split(x_data_train, y_data_train, test_size = 0.2)

#print('Train Set')
#print(x_train, '\n')
#print(y_train, '\n')
#print('Test Set')
##print(x_test, '\n')
#print(y_test)

Train Set
4571    [anndd, relaaxx, call, clonazepam, dead, kenne...
8572       [somebody, pls, get, dj, khaled, tylenol, url]
914     [user, mention, user, mention, user, mention, ...
6767    [spent, six, hours, er, tonight, zofran, morph...
4938    [days, predinsone, covered, hydrocortisone, an...
                              ...                        
4582    [window, boxes, must, steroids, excited, explo...
9933                                [anyone, deck, xanax]
4512    [user, mention, tylenol, new, best, friend, de...
5297    [ok, honestly, morphine, entire, vmas, dont, r...
8245    [user, mention, panadol, please, bakit, kasi, ...
Name: tweet, Length: 7992, dtype: object 

4571    0
8572    0
914     1
6767    1
4938    1
       ..
4582    0
9933    0
4512    0
5297    1
8245    0
Name: label, Length: 7992, dtype: int64 

Test Set
8640                        [target, hours, sleep, xanax]
4823    [user, mention, user, mention, candidate, prop...
3671    [like, take, tylenol, sh

In [ ]:
#Function for getting the maximum review length, by calculating the mean of all the reviews length (using numpy.mean)

In [30]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [ ]:
#Tokenize and Pad/Truncate Reviews
#post, pad or truncate the words in the back of a sentence
#pre, pad or truncate the words in front of a sentence

In [31]:
# ENCODE tweets
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[    2     1   200 ...   944  3624  1952]
 [  115   122   147 ...   193    40   322]
 [    6     3   330 ...   626  1710    29]
 ...
 [    7   529  1739 ...     0     0     0]
 [12658 12659     8 ...     0     0     0]
 [  645  1436   174 ...     0     0     0]] 

Encoded X Test
 [[  98  606  109 ...  193    4  318]
 [ 585   11   56 ...   16  707   55]
 [ 327   12 1209 ...  778    5   88]
 ...
 [   2    1  126 ...    0    0    0]
 [   2    1    3 ...    0    0    0]
 [  59    8    9 ...    0    0    0]] 

Maximum review length:  10


In [ ]:
#Build the LSTM model

In [41]:
#Set hyperparameters
lstm_model .compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [40]:
#LSTM MODEL
embedding_dim = 100

lstm_model = Sequential()
lstm_model.add(Embedding(total_words, embedding_dim, input_length = max_length))
lstm_model.add(LSTM(64, return_sequences=True))
lstm_model.add(LSTM(32))
lstm_model.add(Dense(1, activation='sigmoid'))

lstm_model.build(input_shape=(None, max_length))

print(lstm_model.summary())


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ (None, 10, 100)        │     1,266,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_10 (LSTM)                  │ (None, 10, 64)         │        42,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,320,689 (5.04 MB)

 Trainable params: 1,320,689 (5.04 MB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
#Model Training

In [35]:
checkpoint = ModelCheckpoint(
    'models/LSTM.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

In [ ]:
#Model Training

In [42]:
lstm_model.fit(x_train, y_train, batch_size = 128, epochs =10, callbacks=[checkpoint])

Epoch 1/10
78/79 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.7107 - loss: 0.5968
Epoch 1: accuracy did not improve from 0.97428
79/79 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step - accuracy: 0.7119 - loss: 0.5950
Epoch 2/10
78/79 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.8510 - loss: 0.3539
Epoch 2: accuracy did not improve from 0.97428
79/79 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - accuracy: 0.8510 - loss: 0.3538
Epoch 3/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.9047 - loss: 0.2510
Epoch 3: accuracy did not improve from 0.97428
79/79 ━━━━━━━━━━━━━━━━━━━━ 4s 53ms/step - accuracy: 0.9046 - loss: 0.2511
Epoch 4/10
78/79 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.9291 - loss: 0.1921
Epoch 4: accuracy did not improve from 0.97428
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - accuracy: 0.9290 - loss: 0.1924
Epoch 5/10
77/79 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.9353 - loss: 0.1705
Epoch 5: accuracy did not improve from 0.97428
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - ac

79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - accuracy: 0.9815 - loss: 0.0631


In [44]:
#Model testing

In [43]:
pred = lstm_model.predict(x=x_test)
y_pred = (pred >= 0.5) * 1

#y_pred = model.predict(x_test)

true = 0
for i, y in enumerate(y_test):
    if y == y_pred[i]:
        true += 1


print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {}'.format(true/len(y_pred)*100))

105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
Correct Prediction: 2595
Wrong Prediction: 736
Accuracy: 77.90453317322125
